In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import sys, os, time, math, re
from pathlib import Path
from collections import Counter, defaultdict
import pandas as pd
import numpy as np

# Installing whisper and ffmpeg 
print("Installing openai-whisper and ffmpeg (may take ~20-60s)...")
os.system("pip install --quiet openai-whisper==20240930")
# installing ffmpeg to ensure whisper can process audio files
os.system("apt-get install -y -qq ffmpeg >/dev/null 2>&1")

# imports 
import soundfile as sf
import librosa
import torch
import whisper
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Config 
INPUT_DIR = '/kaggle/input/nppe-2-automatic-disfluency-restoration'
AUDIO_DIR = os.path.join(INPUT_DIR, 'downloaded_audios')
MODEL_SIZE = "tiny"
MAX_TEST_ASR = None    
MAX_TRAIN_RETRIEVE = 1000
VERBOSE = True

Installing openai-whisper and ffmpeg (may take ~20-60s)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.1 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires pylibraft-cu12==25.6.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires rmm-cu12==25.6.*, but you have rmm-cu12 25.2.0 which is incompatible.


In [3]:
# load CSVs and disfluency list
train_csv = os.path.join(INPUT_DIR, 'train.csv')
test_csv  = os.path.join(INPUT_DIR, 'test.csv')
disf_csv  = os.path.join(INPUT_DIR, 'unique_disfluencies.csv')

train_df = pd.read_csv(train_csv) if os.path.exists(train_csv) else pd.DataFrame()
test_df  = pd.read_csv(test_csv)
disf_df  = pd.read_csv(disf_csv) if os.path.exists(disf_csv) else pd.DataFrame()
disf_list = [str(x).strip() for x in (disf_df['disfluency'].astype(str).tolist() if not disf_df.empty else []) if str(x).strip()]
disf_set = set(disf_list)
print("Train rows:", len(train_df), "Test rows:", len(test_df), "unique disfluencies:", len(disf_set))

Train rows: 900 Test rows: 100 unique disfluencies: 29


In [4]:
# ensure train.clean_transcript exists
if not train_df.empty:
    if 'clean_transcript' not in train_df.columns or train_df['clean_transcript'].isnull().any():
        if disf_list:
            import re
            pattern = r"\b(" + "|".join(re.escape(w) for w in disf_list) + r")\b"
            train_df['clean_transcript'] = train_df['transcript'].astype(str).map(lambda s: re.sub(pattern, "", s)).map(lambda s: " ".join(str(s).split()))
        else:
            train_df['clean_transcript'] = train_df['transcript'].astype(str)


In [5]:
# build train-aug seq counts and context (evidence)
def find_insert_sequences(orig_text, clean_text):
    o = orig_text.split(); c = clean_text.split()
    i,j=0,0; inserts=[]
    while i < len(o):
        if j < len(c) and o[i] == c[j]:
            i+=1; j+=1
        else:
            seq=[]
            while i < len(o) and not (j < len(c) and o[i] == c[j]):
                seq.append(o[i]); i+=1
            if seq:
                inserts.append((j, tuple(seq)))
    return inserts

aug_seq_counts = Counter() #count fo disfluencies and what comes before and after it
aug_ctx_counts = defaultdict(Counter)
if not train_df.empty:
    for _, r in train_df.iterrows():
        orig = str(r['transcript']).strip(); clean = str(r['clean_transcript']).strip()
        if not orig: continue
        otoks = orig.split(); L = len(otoks)
        for start, seq in find_insert_sequences(orig, clean):
            aug_seq_counts[seq] += 1
            left = otoks[start-1] if start-1 >= 0 else None
            right_idx = start + len(seq)
            right = otoks[right_idx] if right_idx < L else None
            aug_ctx_counts[(left, right)][seq] += 1
print("Train-derived augmented sequences:", len(aug_seq_counts))


Train-derived augmented sequences: 732


In [6]:
# quick bigram LM for lm_delta checks 
#computes probability of a token sequence to judge whether insertions improve fluency
from collections import defaultdict as _dd, Counter as _Counter
bigram_counts = _dd(_Counter); unigram_counts = _Counter()
if not train_df.empty:
    for s in train_df['transcript'].astype(str).tolist():
        toks = ['<s>'] + s.split() + ['</s>']
        for i in range(len(toks)-1):
            unigram_counts[toks[i]] += 1
            bigram_counts[toks[i]][toks[i+1]] += 1
    unigram_counts['</s>'] += 1
Vocab_size = max(1, len(unigram_counts)); alpha_k = 1.0
def bigram_logprob(tokens):
    toks = ['<s>'] + tokens + ['</s>']; lp=0.0
    for i in range(len(toks)-1):
        w1,w2 = toks[i], toks[i+1]
        c_w1 = unigram_counts.get(w1,0)
        c_w1_w2 = bigram_counts.get(w1,{}).get(w2,0)
        prob = (c_w1_w2 + alpha_k) / (c_w1 + alpha_k * Vocab_size)
        lp += math.log(prob)
    return lp

In [7]:
# load openai-whisper model
print("Loading openai/whisper model:", MODEL_SIZE, "(may take 20-60s)")
device = "cuda" if torch.cuda.is_available() else "cpu"
whisper_model = whisper.load_model(MODEL_SIZE, device=device)

Loading openai/whisper model: tiny (may take 20-60s)


100%|██████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 141MiB/s]


In [8]:
# whisper transcription helper (robust)
#Transcribes audio
#Falls back to safer decoding if errors occur
#ASR can fail on noisy/long audio.
def transcribe_whisper(path, language="hi"):
    try:
        res = whisper_model.transcribe(path, language=language, fp16=(device=="cuda"))
        return res.get('text','').strip()
    except Exception:
        try:
            res = whisper_model.transcribe(path, language=language, temperature=0.0)
            return res.get('text','').strip()
        except Exception:
            return ""

In [9]:
# map test ids -> audio paths robustly
#Matches test IDs to .wav files using multiple heuristics because Audio filenames may not match IDs exactly.
audio_files = sorted([p for p in Path(AUDIO_DIR).glob("*.wav")]) if os.path.exists(AUDIO_DIR) else []
audio_map = {p.stem: str(p) for p in audio_files}
def find_audio(sid):
    s = str(sid)
    if s in audio_map: return audio_map[s]
    try:
        if str(int(s)) in audio_map: return audio_map[str(int(s))]
    except Exception:
        pass
    for k,v in audio_map.items():
        if s in k:
            return v
    return None

test_id_to_audio = {}
missing_audio_ids = []
for sid in test_df['id'].astype(str).tolist():
    p = find_audio(sid)
    if p:
        test_id_to_audio[sid] = p
    else:
        missing_audio_ids.append(sid)
print("Mapped test audio:", len(test_id_to_audio), "Missing audio ids:", len(missing_audio_ids))


Mapped test audio: 100 Missing audio ids: 0


In [10]:
# helper functions: edit distance + find insertions
#Finds tokens ASR says but clean text lacks
#Edit distance helps verify whether an insertion aligns better with spoken audio
def token_edit_distance(a, b):
    a = a.split() if isinstance(a, str) else list(a)
    b = b.split() if isinstance(b, str) else list(b)
    m, n = len(a), len(b)
    if n == 0: return m
    prev = list(range(n+1))
    for i in range(1, m+1):
        cur = [i] + [0]*n
        for j in range(1, n+1):
            cost = 0 if a[i-1] == b[j-1] else 1
            cur[j] = min(prev[j]+1, cur[j-1]+1, prev[j-1]+cost)
        prev = cur
    return prev[n]

def find_insertions(asr_text, clean_text):
    a = asr_text.split(); c = clean_text.split()
    i=j=0; inserts=[]
    while i < len(a) and j < len(c):
        if a[i] == c[j]:
            i+=1; j+=1
        else:
            seq=[]; start_j=j
            while i < len(a) and not (j < len(c) and a[i] == c[j]):
                seq.append(a[i]); i+=1
            inserts.append((start_j, tuple(seq)))
    if i < len(a):
        inserts.append((len(c), tuple(a[i:])))
    return inserts

_deva_re = re.compile(r'[\u0900-\u097F]')
def token_is_deva(tok): return bool(_deva_re.search(tok))

In [11]:
# TF-IDF retrieval index (fallback)
#Builds a retrieval index of training clean transcripts for time when asr evidence is insufficient
tv = None; tv_fit = None
train_texts = []; train_orig_texts = []
if not train_df.empty:
    train_texts = train_df['clean_transcript'].fillna('').astype(str).tolist()
    train_orig_texts = train_df['transcript'].fillna('').astype(str).tolist()
    vec_limit = min(MAX_TRAIN_RETRIEVE, len(train_texts)) if MAX_TRAIN_RETRIEVE else len(train_texts)
    if vec_limit > 0:
        tv = TfidfVectorizer(analyzer='word', token_pattern=r'(?u)\b\w+\b', max_features=20000)
        tv_fit = tv.fit_transform(train_texts[:vec_limit])

In [12]:
# ASR loop: transcribe up to MAX_TEST_ASR and accept insertions conservatively
#Transcribe audio
MAX = MAX_TEST_ASR if (MAX_TEST_ASR is not None) else len(test_id_to_audio)
items = list(test_id_to_audio.items())[:MAX]
print("Will transcribe", len(items), "test audios for ASR evidence.")
start_all = time.time()
accepted_preds = {}
asr_texts = {}
for idx,(sid,path) in enumerate(items,1):
    asr_text = transcribe_whisper(path, language="hi")
    asr_texts[sid] = asr_text
    clean_vals = test_df.loc[test_df['id'].astype(str) == sid, 'transcript'].astype(str).values
    clean = clean_vals[0] if len(clean_vals) > 0 else ""
    pred = clean
    if asr_text and clean:    ##Compare ASR vs clean transcript
        inserts = find_insertions(asr_text, clean) #Find candidate insertions
        accepted=[]
        for pos, seq in inserts:
            if not seq: continue  #accept insertions using rules Because ASR output is noisy, we cannot blindly insert everything it detects.
            # Rule A: contains known disfluency token
            if any(tok in disf_set for tok in seq):
                accepted.append((pos, seq, "disf")); continue
            # Rule B: seen in train and reduces ED to ASR
            freq = aug_seq_counts.get(seq,0)
            if freq >= 1:
                cand = clean.split(); cand2 = cand.copy(); cand2[pos:pos] = list(seq)
                d_before = token_edit_distance(asr_text, clean)
                d_after  = token_edit_distance(asr_text, " ".join(cand2))
                if d_after <= d_before - 0.5:
                    accepted.append((pos, seq, "aug_reduces")); continue
            # Rule C: short Devanagari filler
            if len(seq) <= 2 and all(token_is_deva(t) and len(t) <= 3 for t in seq):
                accepted.append((pos, seq, "short_deva")); continue
        # apply accepted greedily (priority: disf>aug_reduces>short_deva), keep up to 2
        if accepted:
            prio = {"disf":3,"aug_reduces":2,"short_deva":1}
            accepted = sorted(accepted, key=lambda x:(-prio.get(x[2],0), -aug_seq_counts.get(x[1],0)))
            toks = clean.split(); shift=0; used=[]
            for pos, seq, tag in accepted:
                start = pos + shift; end = start + len(seq) - 1
                conflict=False
                for s,e in used:
                    if not (end < s or start > e):
                        conflict=True; break
                if conflict: continue
                toks[start:start] = list(seq)
                used.append((start, start+len(seq)-1)); shift += len(seq)
                if len(used) >= 2: break
            # postprocess collapse repeats and drop long non-deva tokens
            final=[]
            for t in toks:
                if len(final) >= 3 and t == final[-1] == final[-2]:
                    continue
                final.append(t)
            final = [t for t in final if (token_is_deva(t) or len(t) <= 3)]
            pred = " ".join(final)
        else:
            pred = clean
    accepted_preds[sid] = pred
    if idx % 20 == 0 or idx == len(items):
        print(f"ASR processed {idx}/{len(items)} - elapsed {time.time()-start_all:.1f}s")

Will transcribe 100 test audios for ASR evidence.
ASR processed 20/100 - elapsed 46.7s
ASR processed 40/100 - elapsed 93.7s
ASR processed 60/100 - elapsed 164.8s
ASR processed 80/100 - elapsed 234.6s
ASR processed 100/100 - elapsed 273.5s


In [13]:
# Fallback retrieval for remaining test ids (TF-IDF)
#Applies retrieval-based insertion if ASR didn’t help.
final_preds = []
test_ids = test_df['id'].astype(str).tolist()
for sid in test_ids:
    if sid in accepted_preds:
        final_preds.append(accepted_preds[sid]); continue
    clean_vals = test_df.loc[test_df['id'].astype(str) == sid, 'transcript'].astype(str).values
    clean = clean_vals[0] if len(clean_vals) > 0 else ""
    pred = clean
    if tv is not None and tv_fit is not None and clean:
        q = tv.transform([clean])
        sims = cosine_similarity(q, tv_fit).ravel()
        best_idx = sims.argmax()
        if sims[best_idx] > 0.14:
            cand_orig = train_orig_texts[best_idx]
            cand_clean = train_texts[best_idx]
            ins = find_insert_sequences(cand_orig, cand_clean)
            toks = clean.split(); applied=False
            for start, seq in ins:
                if any(tok in disf_set for tok in seq) or aug_seq_counts.get(seq,0) >= 2:
                    left = cand_clean.split()[start-1] if start-1 >= 0 and len(cand_clean.split())>0 else None
                    right_idx = start + len(seq)
                    right = cand_clean.split()[right_idx] if right_idx < len(cand_clean.split()) else None
                    for pos_try in range(0, len(toks)+1):
                        left_ok = (left is None) or (pos_try-1 >= 0 and toks[pos_try-1] == left)
                        right_ok = (right is None) or (pos_try < len(toks) and toks[pos_try] == right)
                        if left_ok and right_ok:
                            toks[pos_try:pos_try] = list(seq); applied=True; break
                if applied: break
            if applied:
                final_tok=[]
                for t in toks:
                    if len(final_tok) >= 3 and t == final_tok[-1] == final_tok[-2]:
                        continue
                    final_tok.append(t)
                final_tok = [t for t in final_tok if (token_is_deva(t) or len(t) <= 3)]
                pred = " ".join(final_tok)
    final_preds.append(pred)

In [14]:
# write submission
out_df = pd.DataFrame({'id': test_ids, 'transcript': final_preds})
out_name = 'submission.csv'
out_df.to_csv(out_name, index=False, encoding='utf-8')
print("Wrote submission:", out_name)
print("Sample 8 outputs:")
for i in range(min(8, len(out_df))):
    sid = out_df.loc[i,'id']
    print("----")
    print("id:", sid)
    print("CLEAN:", test_df.loc[test_df['id'].astype(str)==sid, 'transcript'].values[0])
    print("PRED :", out_df.loc[i,'transcript'])

Wrote submission: submission.csv
Sample 8 outputs:
----
id: 8894265003
CLEAN: जैसे वो दरी वगेरा बना सकते हैं जैसे घर में जो
PRED : जैसे वो दरी वगेरा बना सकते हैं जैसे घर में जो
----
id: 8951729741
CLEAN: क्या
PRED : क्या
----
id: 4268956831
CLEAN: हम आप अपने हूं जो खास दोस्त रहता है उससे लड़ाई की हो
PRED : हम आप अपने हूं जो खास दोस्त रहता है उससे लड़ाई की हो
----
id: 1819728609
CLEAN: सही बात है जिसमें कुछ मोरल हो
PRED : सही बात है जिसमें कुछ मोरल हो
----
id: 5456358058
CLEAN: तनकियों अं तनकियों के बारे में बात करना काम और ब्रैक्स काम ओर ब्रेस के बीच ब्लैं बैलेंस बनाते बनाने की बात करना रोज रोज़ाना के गोल
PRED : तनकियों अं तनकियों के बारे में बात करना काम और ब्रैक्स काम ओर ब्रेस के बीच ब्लैं बैलेंस बनाते बनाने की बात करना रोज रोज़ाना के गोल
----
id: 7463846466
CLEAN: जी जी जी
PRED : जी जी जी
----
id: 1974265135
CLEAN: अच्छा पोर्स हुँ सही है सही रेट अच्छा पर हैड है अच्छा ओके
PRED : अच्छा पोर्स हुँ सही है सही रेट अच्छा पर हैड है अच्छा ओके
----
id: 2894086166
CLEAN: तो करा ै कि मेडम आप देखि